In [1]:
from statsbombpy import sb
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from mplsoccer import Pitch

In [6]:
#Get all competitions
competitions = sb.competitions()

# Get all matches from 2018 Men's World Cup
matches = sb.matches(competition_id=43, season_id=3)

# Choose a Spain match
spain_match = matches[matches['home_team'] == 'Spain'].iloc[0]
match_id = spain_match['match_id']

# Load event data
events = sb.events(match_id=match_id)

competitions.head()

competitions['competition_name'].unique()

array(['1. Bundesliga', 'African Cup of Nations', 'Champions League',
       'Copa America', 'Copa del Rey', "FA Women's Super League",
       'FIFA U20 World Cup', 'FIFA World Cup', 'Indian Super league',
       'La Liga', 'Liga Profesional', 'Ligue 1', 'Major League Soccer',
       'North American League', 'NWSL', 'Premier League', 'Serie A',
       'UEFA Euro', 'UEFA Europa League', "UEFA Women's Euro",
       "Women's World Cup"], dtype=object)